# Tracking options

Exploring the impact of some of the tracking options for the sample datasets.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import tams

%matplotlib inline

xr.set_options(display_expand_data=False)

In [ ]:
tb = tams.load_example_tb()
tb

## Identify cloud elements (CEs)

In [ ]:
%%time

ces, _ = tams.identify(tb)

## Tracking options

In [ ]:
%%time

cases = {}
time = tb.time.values.tolist()

proj = -15

cases["default"] = tams.track(ces, time)
cases["link=f, norm=min"] = tams.track(ces, time, look="f", overlap_norm="min")
cases["link=f, norm=max"] = tams.track(ces, time, look="f", overlap_norm="max")
cases[f"u_proj={proj}"] = tams.track(ces, time, u_projection=proj)
cases[f"u_proj={proj} look=f, norm=min"] = tams.track(ces, time, u_projection=proj, look="f", overlap_norm="min")
cases[f"u_proj={proj} look=f, norm=max"] = tams.track(ces, time, u_projection=proj, look="f", overlap_norm="max")

for key, ce in cases.items():
    cases[key] = tams.classify(ce)

In [ ]:
m, n = 2, 3
assert len(cases) <= m * n

# Restrict range
lon_min, lat_min, lon_max, lat_max = -10, 8, -3, 15
lon_min, lat_min, lon_max, lat_max = 17, 1, 27, 8
lon_min, lat_min, lon_max, lat_max = 20, 1, 27, 4
lon_min, lat_min, lon_max, lat_max = 30, 10, 38, 16

proj = ccrs.PlateCarree()

fig, axs = plt.subplots(m, n, figsize=(8, 5), sharex=True, sharey=True, subplot_kw=dict(projection=proj), constrained_layout=True)

for i, ((key, ce), ax) in enumerate(zip(cases.items(), axs.flat)):
    if i == 0:
        assert key == "default"
        tams.plot_tracked(ce)

    tams.plot_tracked(ce.cx[lon_min:lon_max, lat_min:lat_max], ax=ax)
    ax.text(0.03, 0.98, key, ha="left", va="top", transform=ax.transAxes, size=8)
    gl = ax.gridlines(draw_labels=True, color="none")
    if not i < n:
        gl.top_labels = False
    if not i % n == 0:
        gl.left_labels = False